In [1]:
import os
import numpy as np
import cv2
from skimage.io import imread
from skimage.io import imsave
from skimage.util import random_noise
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
from skimage.color import rgb2gray

C:\Users\wyw\Miniconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
def AugmentImage(img, folder_name, image_prefix, N):
    
    """ This function uses the keras ImageDataGenerator for image augmentation.
    
    Arguments:
    
    img : Image that needs to be augmented
    folder_name : name of the folder where the images will be saved
    image_prefix : prefix for the saved images
    N : number of images that will be created through augmentation
    
    Returns:
    
    N augmented images with 'image_prefix' saved in 'folder_name'
    
    Raises:
    
    Error when input data type is incorrect
    
    """
    
    assert type(folder_name) == str, ('Wrong Data type', 'folder_name must be a string')
    assert type(image_prefix) == str, ('Wrong Data type', 'image_prefix must be a string')
    assert type(N) == int, ('Wrong Data type', 'N must be an integer')
    
    directory = './Multislice'+"/"+image_prefix+"/"+folder_name
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    datagen = ImageDataGenerator(rotation_range=10,
                                 shear_range=0.2,
                                 zoom_range=[0.7,1],
                                 rescale=1.0/225,
                                 horizontal_flip=True, 
                                 vertical_flip=True, 
                                 fill_mode='constant', 
                                 cval = 0.0)

    
    x = img_to_array(img)  # this is a Numpy array with shape (64, 64, 1)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 64, 64, 1)

    i = 0
    #save the image data into each type of defect directory 
    for batch in datagen.flow(x, batch_size = 1,
                              save_to_dir = './Multislice'+'/'+image_prefix+'/'+folder_name, save_prefix = image_prefix, save_format = 'png'):
        i += 1
        if i > N:
            break  # otherwise the generator would loop indefinitely
    
    return

In [18]:
def AddNoiseImage(img, folder_name, image):
    
    #img is the augemented data you read in
    #folder_name is the name of the folder you want to store all the noisy images
    #image prefix: defect type...
    
    assert type(folder_name) == str, ('Wrong Data type', 'folder_name must be a string')
    assert type(image) == str, ('Wrong Data type', 'image_prefix must be a string')
    
    directory = './Multislice' +'/'+ folder_name+'/'+folder_name+'_Augmented'+'/'+folder_name+'_noise'
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    img_grey=rgb2gray(img)
    img_gauss=random_noise(img_grey,mode='gaussian',mean=0.,var=0.01)
    img_sp=random_noise(img_grey,mode='s&p',salt_vs_pepper=0.5)
    img_poisson=random_noise(img_grey,mode='poisson')
    
    imsave(directory+'/'+'gauss_'+image,img_gauss)
    imsave(directory+'/'+'s&p_'+image,img_sp)
    imsave(directory+'/'+'poisson'+image,img_poisson)
    
    return 

In [19]:
# the path of the folder includes different type of defects (Multislice)
path = "./Multislice"
folders=os.listdir(path) # get all subfolders' names (all types of defects' names)
for folder in folders:  #traversing all folders(types of defects) in the folder of MULTISLICE
    path = "./Multislice"+"/"+folder
    files=os.listdir(path)
    for file in files: #traversing all raw_image in the folder of different defect types
        if file not in (folder+"_Augmented",'.DS_Store'):
            img=Image.open(path+'/'+file).convert('L') # loading the image
            # Augmenting using the function above
            AugmentImage(img, folder+'_Augmented', folder, 200)
    # the path of data_Augmented folder
    noisy_path = "./Multislice"+"/"+folder+'/'+folder+'_Augmented'
    # get all augmented image
    image=os.listdir(noisy_path)
    for i in image:
        if i not in ('noisydata','.DS_Store'):
            img=imread(noisy_path+'/'+i)
            AddNoiseImage(img, folder, i)
            
        else: 
            pass
        
       

C:\Users\wyw\Miniconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


In [6]:
image=os.listdir("./Multislice/Mo/Mo_Augmented")

In [8]:
for i in image:
    img=imread("./Multislice/Mo/Mo_Augmented"+'/'+i)

In [10]:
img.shape

(64, 64)